<a href="https://colab.research.google.com/github/shriver42/ESAA/blob/%ED%95%84%EC%82%AC-%EA%B3%BC%EC%A0%9C/1014_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CH9 추천 시스템



###01 추천 시스템의 개요와 배경
- 유형:
 - 콘텐츠 기반 필터링 방식
 - 협업 필터링 방식

###02 콘텐츠 기반 필터링 추천 시스템
: 사용자가 선호하는 아이템을 기반으로 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식

###03 최근접 이웃 협업 필터링
: 사용자 행동 양식(User Behavior)만을 기반으로 추천을 수행하는 방식
- 목표: 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 에측 평가하는 것
- 방식:
  - 최근접 이웃 방식(메모리 협업 필터링)
    - 사용자 기반: 사용자 기반 최근접 이웃 방식은 특정 사용자와 유사한 다른 사용자를 Top-N으로 선정해 이
Top-N 사용자가 좋아하는 아이템을 추천하는 방식

    - 아이템 기반: 아이템이 가지는
속성과는 상관없이 사용자들이 그 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템을 추천하는 기준이 되는 알고리즘, 대부분 이 방식을 사용함.
  - 잠재 요인 방식
  - 추천 시스템의 유사도 측정에 코사인 유사도 사용됨.

##04. 잠재 요인 협업 필터링
- 잠재 요인 협업 필터링: 사용자-아이템 평점 행렬 데이터만을 이용해 말 그대로 ‘잠재 요인’을 끄집어내는 것을 의미
- ‘잠재 요인’을 기반으로 다차원 희소 행렬인 사용자一아이템 행렬 데이터를 저차원 밀집 행렬의 사용자-잠재
요인 행렬과 아이템-잠재 요인 행렬의 전치 행렬（즉, 잠재 요인-아이템 행렬）로 분해할 수 있으며, 이렇게 분해된 두 행렬의 내적을 통해 새로운 예측 사용자—아이템 평점 행렬 데이터를 만들어서 사용자가 아직 평점을 부여하지 않는 아이템에 대한 예측 평점을 생성하는 방식

**행렬 분해의 이해**
- 다차원의 매트릭스를 저차원 매트릭스로 분해하는 기법
  -  SVD(Singular Vector Decomposition), NMF(Non—Negative Matrix Factorization)
- M개의 사용자행과 N개의 아이템열을 가진 평점 행렬 R은 M X N 차원으로 구성
  - 행렬 분해를 통해서 사용자-K 차원 잠재 요인 행렬 P(MXK 차원）와 K 차원 잠재 요인 - 아이템 행렬 Q.T(KXN 차원）로 분해됨.(Q 는 아이템-잠재 요인 행렬, Q.T 는 Q 의 전치 행렬인 잠재 요인-아이템 행렬)

**확률적 경사 하강법을 이용한 행렬 분해**
- P 와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것
1. P와 Q를 임의의 값을 가진 행렬로 설정
2. P와 Q.T 값을 곱해 예측 R 행렬을 계산하고 예측 R 행렬과 실제 R 행렬에 해당하는 오류 값을 계산
3. 이 오류 값을 최소화할 수 있도록 P와 Q 행렬을 적절한 값으로 각각 업데이트
4. 만족할 만한 오류 값을 가질 때까지 2, 3번 작업을 반복하면서 P와 Q 값을 업데이트해 근사화
- 일반적으로 사용자-아이템 평점 행렬의 경우 행렬 분해를 위해서 단순히 예측 오류값의 최소화와 학습 시 과적합을 피하기 위해서 규제를 반영한 비용 함수를 적용함.
- SDG 기반의 행렬 분해: L2 규제를 반영해 실제 R 행렬 값과 예측 R 행렬 값의 차이를 최소화하는 방향성을 가지고 P행렬과 Q행렬에 업데이트 값을 반복적으로 수행하면서 최적화된 예측 R 행렬을 구하는 방식


In [ ]:
import numpy as np

#원본 행렬 R 생성, 분해 행렬 이와 Q 초기화, 잠재 요인 차원 유는 3으로 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K=3

#P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    #두 개의 분해된 행렬 으와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    #실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse


In [ ]:
#R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장.
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

steps=1000
learning_rate=0.01
r_lambda=0.01

#SGD 기법으로 우와 Q 매트릭스를 계속 업데이트.
for step in range(steps):
  for i, j, r in non_zeros:
    #실제 값과 예측 값의 차이인 오류 값 구함
    eij = r - np.dot(P[i, :], Q[j, :].T)
    #Regularization을 반영한 SGD 업데이트 공식 적용
    P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
    Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
      print("### iteration step : ", step, " rmse : ", rmse)

### iteration step :  0  rmse :  0.016599129218140208
### iteration step :  0  rmse :  0.01668830060775919
### iteration step :  0  rmse :  0.016548113949791696
### iteration step :  0  rmse :  0.01647540947076357
### iteration step :  0  rmse :  0.01658935692516356
### iteration step :  0  rmse :  0.016603539486165532
### iteration step :  0  rmse :  0.016426936856238406
### iteration step :  0  rmse :  0.016404876258812897
### iteration step :  0  rmse :  0.01608305843990478
### iteration step :  0  rmse :  0.0162146619204775
### iteration step :  0  rmse :  0.01630027086162528
### iteration step :  0  rmse :  0.01655803890814364
### iteration step :  50  rmse :  0.016555532685664495
### iteration step :  50  rmse :  0.01664375695415199
### iteration step :  50  rmse :  0.016502704735255096
### iteration step :  50  rmse :  0.01643111453746046
### iteration step :  50  rmse :  0.01654535507021205
### iteration step :  50  rmse :  0.016559308517824143
### iteration step :  50  rmse : 

In [ ]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 1.096 1.291 2.001 1.695]
 [6.267 4.978 0.878 2.982 1.003]
 [6.551 0.886 2.987 3.978 3.986]
 [4.969 2.005 1.007 2.013 1.276]]
